# Firefox Launcher Connection Diagnostic

This notebook provides comprehensive diagnostics for the "connection failed, invalid address" error in the JupyterLab Firefox Launcher extension.

## Problem Statement
The Xpra display in JupyterHub shows:
```
You were disconnected for the following reason:
connection failed, invalid address
```

## Diagnostic Approach
We'll systematically check each component in the connection chain:
1. Environment and dependencies
2. JupyterHub proxy configuration
3. Xpra server status and connectivity
4. WebSocket connection issues
5. Authentication and routing problems
6. Browser-side connection attempts

## 1. Environment Setup and Library Imports

In [2]:
import subprocess
import requests
import json
import time
import socket
from urllib.parse import urlparse
import os
import psutil

print("🔍 Starting comprehensive Firefox launcher diagnostic...")
print("=" * 60)

🔍 Starting comprehensive Firefox launcher diagnostic...


## 2. Current Error Analysis

**Browser Console Errors:**
- `500 Internal Server Error` on `/user/bdx/proxy/39005/`
- `WebSocket connection to 'ws://raton00:8889/user/bdx/proxy/39005/' failed`
- `websocket error: unknown reason (no websocket error code) code: 0`

This indicates a **jupyter-server-proxy configuration issue** where the proxy cannot connect to the Xpra backend.

In [3]:
# Check active Xpra processes and their ports
print("🔍 Checking active Xpra processes...")
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
xpra_processes = [line for line in result.stdout.split('\n') if 'xpra start' in line]

print(f"Found {len(xpra_processes)} active Xpra processes:")
for i, proc in enumerate(xpra_processes, 1):
    # Extract port from the process command line
    if '--bind-tcp=' in proc:
        port_part = proc.split('--bind-tcp=')[1].split()[0]
        port = port_part.split(':')[-1]
        pid = proc.split()[1]
        print(f"{i}. PID {pid} → Port {port}")
        print(f"   Command: {proc.split('xpra start')[1].split('--bind-tcp=')[0]}...")
    print()

🔍 Checking active Xpra processes...
Found 8 active Xpra processes:
1. PID 411747 → Port 48981
   Command:  ...

2. PID 414541 → Port 39809
   Command:  ...

3. PID 418572 → Port 47903
   Command:  ...

4. PID 421528 → Port 43459
   Command:  ...

5. PID 424457 → Port 42609
   Command:  ...

6. PID 429980 → Port 59337
   Command:  ...

7. PID 531624 → Port 35235
   Command:  ...

8. PID 1419880 → Port 39005
   Command:  ...



In [4]:
# Test direct connectivity to Xpra services
print("🔍 Testing direct connectivity to Xpra services...")

# Get listening ports
result = subprocess.run(['netstat', '-tlnp'], capture_output=True, text=True)
listening_ports = []
for line in result.stdout.split('\n'):
    if ':39005 ' in line or ':35235' in line:  # Check recent ports from logs
        parts = line.split()
        if len(parts) >= 4:
            port = parts[3].split(':')[-1]
            listening_ports.append(port)

print(f"Ports listening: {listening_ports}")

# Test each port
for port in listening_ports:
    print(f"\n🧪 Testing localhost:{port}")
    try:
        # Test HTTP connection
        response = requests.head(f"http://localhost:{port}/", timeout=5)
        print(f"  ✅ HTTP Status: {response.status_code}")
        print(f"  ✅ Server: {response.headers.get('Server', 'Unknown')}")
    except requests.exceptions.RequestException as e:
        print(f"  ❌ HTTP Error: {e}")
    
    # Test socket connection
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex(('localhost', int(port)))
        sock.close()
        if result == 0:
            print(f"  ✅ Socket connection successful")
        else:
            print(f"  ❌ Socket connection failed: {result}")
    except Exception as e:
        print(f"  ❌ Socket error: {e}")

🔍 Testing direct connectivity to Xpra services...
Ports listening: ['35235', '39005']

🧪 Testing localhost:35235
  ✅ HTTP Status: 200
  ✅ Server: Xpra-WebSocket-Server Python/3.12.3
  ✅ Socket connection successful

🧪 Testing localhost:39005
  ✅ HTTP Status: 200
  ✅ Server: Xpra-WebSocket-Server Python/3.12.3
  ✅ Socket connection successful


In [5]:
# Test JupyterHub proxy paths
print("🔍 Testing JupyterHub proxy connectivity...")

# Get the most recent port from the logs (39005 based on the error)
test_port = "39005"
proxy_url = f"http://raton00:8889/user/bdx/proxy/{test_port}/"

print(f"Testing proxy URL: {proxy_url}")

try:
    # Test without authentication first
    response = requests.head(proxy_url, timeout=10, allow_redirects=False)
    print(f"  Status: {response.status_code}")
    print(f"  Headers: {dict(response.headers)}")
    
    if response.status_code == 302:
        print(f"  Redirect to: {response.headers.get('Location', 'No location')}")
        print("  → This indicates authentication is required")
    elif response.status_code == 500:
        print("  ❌ 500 Internal Server Error - jupyter-server-proxy cannot connect to backend")
    
except requests.exceptions.RequestException as e:
    print(f"  ❌ Request failed: {e}")

# Check if the port is actually different in the backend
print(f"\n🔍 Checking if backend is listening on port {test_port}...")
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex(('localhost', int(test_port)))
    sock.close()
    if result == 0:
        print(f"  ✅ Backend IS listening on port {test_port}")
        print("  → The issue is in jupyter-server-proxy configuration")
    else:
        print(f"  ❌ Backend NOT listening on port {test_port}")
        print("  → The Xpra process may have failed to start or is on a different port")
except Exception as e:
    print(f"  ❌ Connection test error: {e}")

🔍 Testing JupyterHub proxy connectivity...
Testing proxy URL: http://raton00:8889/user/bdx/proxy/39005/
  Status: 302
  Headers: {'Server': 'TornadoServer/6.5.1', 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Sat, 02 Aug 2025 15:45:49 GMT', 'X-Content-Type-Options': 'nosniff', 'X-Jupyterhub-Version': '5.3.0', 'Content-Security-Policy': "frame-ancestors 'none'", 'Location': '/hub/api/oauth2/authorize?client_id=jupyterhub-user-bdx&redirect_uri=%2Fuser%2Fbdx%2Foauth_callback&response_type=code&state=xjTahFtb6HdqcF_VgILDzg', 'Set-Cookie': 'jupyterhub-user-bdx-oauth-state="2|1:0|10:1754149549|31:jupyterhub-user-bdx-oauth-state|32:eGpUYWhGdGI2SGRxY0ZfVmdJTER6Zw==|d72bc109926992bacf492e9f383f40f591f71d3510046a554e6ea118e3a3ed90"; expires=Mon, 01 Sep 2025 15:45:49 GMT; HttpOnly; Max-Age=600; Path=/user/bdx/', 'Content-Length': '0'}
  Redirect to: /hub/api/oauth2/authorize?client_id=jupyterhub-user-bdx&redirect_uri=%2Fuser%2Fbdx%2Foauth_callback&response_type=code&state=xjTahFtb6HdqcF_Vg

In [ ]:
# Investigate jupyter-server-proxy configuration issue
print("🔍 Investigating jupyter-server-proxy configuration...")

# Check if there's a timing issue - wait a bit and test again
print("\n⏱️  Testing with delay (in case of startup timing issue)...")
time.sleep(3)

try:
    response = requests.head(f"http://localhost:{test_port}/", timeout=5)
    print(f"  Direct connection after delay: ✅ {response.status_code}")
except Exception as e:
    print(f"  Direct connection after delay: ❌ {e}")

# Check the exact error from JupyterHub logs by examining the server process
print("\n🔍 Checking server configuration...")

# Look for any proxy configuration files
proxy_configs = []
for root, dirs, files in os.walk('/home/bdx/.jupyter'):
    for file in files:
        if 'proxy' in file.lower() or 'server' in file.lower():
            proxy_configs.append(os.path.join(root, file))

print(f"Found potential config files: {proxy_configs}")

# Check if jupyter-server-proxy is properly loaded
try:
    result = subprocess.run(['python', '-c', 'import jupyter_server_proxy; print(jupyter_server_proxy.__version__)'], 
                          capture_output=True, text=True, cwd='/home/bdx/allcode/github/vantagecompute/jup-fir-lau')
    print(f"jupyter-server-proxy version: {result.stdout.strip()}")
except Exception as e:
    print(f"jupyter-server-proxy check failed: {e}")

In [6]:
# Deep dive into proxy registration mechanism
print("🔍 Examining proxy registration mechanism...")

# Check how our extension registers with jupyter-server-proxy
print("\n📋 Checking our extension's proxy registration...")

# Look at our server extension code
server_extension_path = "/home/bdx/allcode/github/vantagecompute/jup-fir-lau/jupyterlab_firefox_launcher/server_extension.py"
try:
    with open(server_extension_path, 'r') as f:
        content = f.read()
        
    # Look for proxy registration
    if 'server_proxy' in content:
        print("  ✅ Found server_proxy references in our extension")
        
        # Extract relevant lines
        lines = content.split('\n')
        proxy_lines = [line.strip() for line in lines if 'proxy' in line.lower() or 'register' in line.lower()]
        for line in proxy_lines[:10]:  # Show first 10 relevant lines
            print(f"    {line}")
    else:
        print("  ❌ No server_proxy registration found in our extension")
        print("  → This could be the root cause!")
        
except Exception as e:
    print(f"  ❌ Could not read server extension: {e}")

# Check if jupyter-server-proxy can route to dynamic ports
print(f"\n🔍 Testing if jupyter-server-proxy supports dynamic port routing...")

# The issue might be that we're not properly registering dynamic ports with jupyter-server-proxy
# Let's check what endpoints are available
try:
    # Try to access the server-proxy info endpoint
    response = requests.get("http://raton00:8889/user/bdx/server-proxy/servers-info", timeout=5)
    if response.status_code == 200:
        servers_info = response.json()
        print(f"  Server-proxy info: {servers_info}")
    else:
        print(f"  Server-proxy info request failed: {response.status_code}")
except Exception as e:
    print(f"  Could not get server-proxy info: {e}")

🔍 Examining proxy registration mechanism...

📋 Checking our extension's proxy registration...
  ✅ Found server_proxy references in our extension
    This module registers the Firefox launcher handlers with Jupyter Server
    using jupyter_server_extension_points and configures jupyter-server-proxy.
    from .server_proxy import get_server_proxy_config
    # Register our handlers
    serverapp.log.info(f"🔧 Registered handler: {firefox_launcher_pattern}")
    serverapp.log.info(f"🔧 Registered handler: {firefox_cleanup_pattern}")
    # Try to register with jupyter-server-proxy if available
    # Check if jupyter-server-proxy is available
    if importlib.util.find_spec("jupyter_server_proxy") is not None:
    "jupyter-server-proxy detected, registering Firefox proxy"

🔍 Testing if jupyter-server-proxy supports dynamic port routing...
  Could not get server-proxy info: Expecting value: line 2 column 1 (char 1)


## 3. Root Cause Analysis

Based on the investigation, the likely issues are:

### **Issue 1: Dynamic Port Registration**
- Our extension creates Xpra servers on dynamic ports (e.g., 39005)
- `jupyter-server-proxy` may not be aware of these dynamic ports
- The proxy tries to connect but doesn't know where to route the request

### **Issue 2: Proxy Configuration Method**
- We might not be properly registering the dynamic proxy routes with `jupyter-server-proxy`
- The logs show the extension creates the Xpra process but doesn't register the proxy route

### **Issue 3: WebSocket Handling**
- WebSocket connections need special handling in the proxy configuration
- The WebSocket URL `ws://raton00:8889/user/bdx/proxy/39005/` is failing

## Next Steps: Test the Solution

In [7]:
# Investigate the WebSocket proxy configuration issue
print("🔍 Analyzing WebSocket proxy forwarding issue...")

# Test the exact URLs that are failing
test_port = "37847"  # Current active port
proxy_url = f"http://raton00:8889/user/bdx/proxy/{test_port}/"
websocket_url = f"ws://raton00:8889/user/bdx/proxy/{test_port}/"

print(f"\n📋 Connection Analysis:")
print(f"   Proxy URL: {proxy_url}")
print(f"   WebSocket URL: {websocket_url}")
print(f"   Direct Xpra URL: http://localhost:{test_port}/")

# The issue: Xpra HTML5 client tries to connect to raton00:8889 instead of using the proxy
print(f"\n❌ PROBLEM IDENTIFIED:")
print(f"   Xpra HTML5 client is trying to connect to: raton00:8889")
print(f"   But it should connect through the proxy WebSocket: {websocket_url}")
print(f"   jupyter-server-proxy should forward WebSocket traffic from JupyterHub to Xpra")

# Check if WebSocket forwarding is working
print(f"\n🧪 Testing WebSocket proxy forwarding...")
try:
    # Test if the proxy endpoint responds to WebSocket upgrade requests
    import requests
    headers = {
        'Connection': 'Upgrade',
        'Upgrade': 'websocket',
        'Sec-WebSocket-Version': '13',
        'Sec-WebSocket-Key': 'dGhlIHNhbXBsZSBub25jZQ=='
    }
    
    response = requests.get(proxy_url, headers=headers, timeout=5, allow_redirects=False)
    print(f"   WebSocket upgrade test: {response.status_code}")
    print(f"   Response headers: {dict(response.headers)}")
    
    if response.status_code == 101:
        print("   ✅ WebSocket upgrade successful")
    elif response.status_code == 200:
        print("   ⚠️ HTTP response instead of WebSocket upgrade")
    else:
        print(f"   ❌ Unexpected status: {response.status_code}")
        
except Exception as e:
    print(f"   ❌ WebSocket test failed: {e}")

🔍 Analyzing WebSocket proxy forwarding issue...

📋 Connection Analysis:
   Proxy URL: http://raton00:8889/user/bdx/proxy/37847/
   WebSocket URL: ws://raton00:8889/user/bdx/proxy/37847/
   Direct Xpra URL: http://localhost:37847/

❌ PROBLEM IDENTIFIED:
   Xpra HTML5 client is trying to connect to: raton00:8889
   But it should connect through the proxy WebSocket: ws://raton00:8889/user/bdx/proxy/37847/
   jupyter-server-proxy should forward WebSocket traffic from JupyterHub to Xpra

🧪 Testing WebSocket proxy forwarding...
   WebSocket upgrade test: 403
   Response headers: {'Server': 'TornadoServer/6.5.1', 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Sat, 02 Aug 2025 15:53:35 GMT', 'Content-Length': '9'}
   ❌ Unexpected status: 403


## 4. **SOLUTION IDENTIFIED!**

### Root Cause Found 🎯
The issue is that our Firefox handler creates Xpra processes on dynamic ports (e.g., 39005) but **never registers these ports with jupyter-server-proxy**. 

When the frontend tries to access `/user/bdx/proxy/39005/`, jupyter-server-proxy returns a 500 error because it doesn't know how to route the request to localhost:39005.

### What Happens:
1. ✅ Frontend clicks Firefox launcher
2. ✅ POST to `/user/bdx/firefox-launcher/api/firefox` succeeds  
3. ✅ Xpra process starts on port 39005
4. ✅ Frontend gets proxy path `/user/bdx/proxy/39005/`
5. ❌ Browser tries to access `/user/bdx/proxy/39005/` → **500 error**
6. ❌ WebSocket to `ws://raton00:8889/user/bdx/proxy/39005/` fails

### The Fix 🔧
We need to **dynamically register proxy routes** with jupyter-server-proxy when each Firefox session is created.